In [70]:
# 2025/6/25
# zhangzhong
# chatbot

In [71]:
# I do not want to use langgraph actually
# I want to try graphrag

In [72]:
from utils import load_env
load_env()

In [73]:
# https://python.langchain.com/docs/integrations/chat/deepseek/
from langchain_deepseek import ChatDeepSeek


model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


In [74]:
# conversation最本质的点其实就是把之前的聊天记录一起发给大模型而已

In [75]:
# LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.
# 相比于之前版本的embedding + vector db. langgraph用了图，可以更好的处理多轮对话，之前是DAG，无环的，不好处理重试这样的逻辑
# langgraph可以带环，可以重复，大概是这样理解吧，现在还不是特别理解，先构建一个例子，然后再去研究底层吧

In [76]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# class MessagesState(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]
# Define a new graph
# A graph whose nodes communicate by reading and writing to a [shared] state
# 所以每个节点过去之后，saver都会更新同一个messages state，也就是往里append消息
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
# 所以，这里的call model传入的消息会越来越多，因为每次调用都会把之前的消息都传进去
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# define the single node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
# # The checkpointer saves the state after each node
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# STATE in LangGraph represents the current data and context that flows through your workflow graph.
# {
#     "messages": [
#         HumanMessage("Hello!"),
#         AIMessage("Hi there! How can I help you?"),
#         HumanMessage("What's the weather like?"),
#         # ... more messages
#     ]
# }

In [77]:
config = {"configurable": {"thread_id": "abc123"}}
# This enables us to support multiple conversation threads with a single application

In [78]:
from langchain_core.messages import HumanMessage

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
for message in output["messages"]:
    message.pretty_print()  # output contains all messages in state

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

Hi Bob! 👋 How's it going? What can I do for you today? 😊


In [79]:
config = {"configurable": {"thread_id": "asdf"}}

input_messages = [HumanMessage("what is my name")]
output = app.invoke({"messages": input_messages}, config)
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

what is my name
================================== Ai Message ==================================

I don’t have access to personal information unless you share it with me. Could you tell me your name? I’d be happy to address you by it! 😊


In [80]:
# Async function for node: I like Async!
# async def call_model(state: MessagesState):
#     response = await model.ainvoke(state["messages"])
#     return {"messages": response}


# # Define graph as before:
# workflow = StateGraph(state_schema=MessagesState)
# workflow.add_edge(START, "model")
# workflow.add_node("model", call_model)
# app = workflow.compile(checkpointer=MemorySaver())

# # Async invocation:
# output = await app.ainvoke({"messages": input_messages}, config)
# output["messages"][-1].pretty_print()

In [81]:
# https://python.langchain.com/docs/tutorials/chatbot/#prompt-templates
# Prompt Templates help to turn raw user information into a format that the LLM can work with.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [82]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [83]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi! I'm Jim.
================================== Ai Message ==================================

Ahoy there, Jim! *adjusts eye patch* Welcome aboard me ship o' words! What brings ye to these treacherous waters today? Be ye lookin' for tales o' adventure, answers to yer burnin' questions, or just a friendly chat with a salty ol' pirate like meself? *leans in, wooden leg creaking* 

Ye've got me full attention, matey - unless, o' course, the rum's gone missin' again. *winks* 

Now speak yer mind, or prepare to walk the plank o' curiosity! Arr!


In [84]:
# the key needed by the prompt template
# should be stored in the state
# so we need to define a new sate ourself

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [85]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [86]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

for message in output["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi! I'm Bob.
================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo estás? ¿En qué puedo ayudarte hoy? 😊


In [87]:
query = "What is my name?"

# Note that the entire state is persisted, so we can omit parameters like language if no changes are desired
# 这个点挺好的！也就是说，saver实际上采取的[合并策略]，而不是覆盖策略
# 包括list是append，dict是merge
input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Tu nombre es **Bob**! 😊 ¿Hay algo más en lo que pueda ayudarte, Bob?


In [88]:
# https://python.langchain.com/docs/tutorials/chatbot/#managing-conversation-history

# 1. load previous messages from Message
# 2. truncate the messages to the last 10 messages
# 3. add to prompt template

from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.messages import HumanMessage, AIMessage

# trimmer = trim_messages(
#     max_tokens=65,
#     strategy="last",
#     token_counter=model,
#     include_system=True,
#     allow_partial=False,
#     start_on="human",
# )

# messages = [
#     SystemMessage(content="you're a good assistant"),
#     HumanMessage(content="hi! I'm bob"),
#     AIMessage(content="hi!"),
#     HumanMessage(content="I like vanilla ice cream"),
#     AIMessage(content="nice"),
#     HumanMessage(content="whats 2 + 2"),
#     AIMessage(content="4"),
#     HumanMessage(content="thanks"),
#     AIMessage(content="no problem!"),
#     HumanMessage(content="having fun?"),
#     AIMessage(content="yes!"),
# ]

# trimmer.invoke(messages)

In [89]:
# 这种方式也挺好的，这应该是最简单的方式

# workflow = StateGraph(state_schema=State)


# def call_model(state: State):
#     trimmed_messages = trimmer.invoke(state["messages"])
#     prompt = prompt_template.invoke(
#         {"messages": trimmed_messages, "language": state["language"]}
#     )
#     response = model.invoke(prompt)
#     return {"messages": [response]}


# workflow.add_edge(START, "model")
# workflow.add_node("model", call_model)

# memory = MemorySaver()
# app = workflow.compile(checkpointer=memory)

In [90]:
# https://python.langchain.com/docs/tutorials/chatbot/#streaming

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|'s| a| joke| for| you|:|  

|**|Why| don|’|t| skeletons| fight| each| other|?|**|  
|*|Because| they| don|’|t| have| the| guts|!|*|  

|Hope| that| gives| you| a| chuckle|!| Let| me| know| if| you|'d| like| another| one|.| 😄||